In [1]:
from glob import glob 
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# from albumentations import HorizontalFlip, GridDistortion, OpticalDistortion, ChannelShuffle, CoarseDropout, CenterCrop, Crop, Rotate
from PIL import Image 

# def load_data(path, split=0.2):
# #     path = r""
#     X = sorted(glob(os.path.join(path, "images", "*.jpg")))
#     Y = sorted(glob(os.path.join(path, "masks", "*.png")))
#     train_x, test_x = train_test_split(X, test_size=0.2, random_state=42)
#     train_y, test_y = train_test_split(Y, test_size=0.2, random_state=42)
#     return (train_x, train_y), (test_x, test_y)
# p = r"/home/jai201/Jai_Minor/semantic_resized"
# load_data(p)

# Augmentation

In [2]:


def augment_data(images, masks, save_path, augment=True):
    H = 512
    W = 512
    for x, y in tqdm(zip(images, masks), total=len(images)):
        """ Extract the name """
        name = os.path.splitext(os.path.basename(x))[0]
        

        """ Reading the image and mask """
        c = cv2.imread(x, cv2.IMREAD_COLOR)
        d = cv2.imread(y, cv2.IMREAD_COLOR)
        x = Image.open(x)
        y = Image.open(y)
        
#         x = np.array(a)
#         y = np.array(b)
        """ Augmentation """
        if augment == True:
            
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=np.array(x), mask=np.array(y))
            x1 = augmented["image"]
            y1 = augmented["mask"]

            x2 = cv2.cvtColor(c, cv2.COLOR_RGB2GRAY)
            y2 = d

            aug = ChannelShuffle(p=1)
            augmented = aug(image=np.array(x), mask=np.array(y))
            x3 = augmented['image']
            y3 = augmented['mask']

            aug = CoarseDropout(p=1, min_holes=3, max_holes=10, max_height=32, max_width=32)
            augmented = aug(image=np.array(x), mask=np.array(y))
            x4 = augmented['image']
            y4 = augmented['mask']

            aug = Rotate(limit=45, p=1.0)
            augmented = aug(image=np.array(x), mask=np.array(y))
            x5 = augmented["image"]
            y5 = augmented["mask"]

            X = [x, x1, x2, x3, x4, x5]
            Y = [y, y1, y2, y3, y4, y5]
            index = 0
            for i, m in zip(X, Y):
                try:
                    """ Center Cropping """
                    aug = CenterCrop(H, W, p=1.0)
                    augmented = aug(image=i, mask=m)
                    i = augmented["image"]
                    m = augmented["mask"]

                except Exception as e:
                    pass
#                     basewidth = 512
# #                     img = Image.open('somepic.jpg')
#                     wpercent = (basewidth/float(Image.fromarray(augmented["image"]).size[0]))
#                     hsize = int((float(Image.fromarray(augmented["image"]).size[1])*float(wpercent)))
#                     im1 = Image.fromarray(augmented["image"]).resize((basewidth,hsize), Image.ANTIALIAS)
#                     im2 = Image.fromarray(augmented["mask"]).resize((basewidth,hsize), Image.ANTIALIAS)
#                     size = 512 , 512
                    
#                     i = im.thumbnail(size, Image.ANTIALIAS)
#                     m = cv2.resize(i , (W,H))
                    

                tmp_image_name = f"{name}_{index}"
                tmp_mask_name = f"{name}_{index}"

                image_path = os.path.join(save_path, "image", tmp_image_name)
                mask_path = os.path.join(save_path, "mask", tmp_mask_name)

    #             cv2.imwrite(image_path, i)
    #             cv2.imwrite(mask_path, m)
                im1 = Image.fromarray(augmented["image"])
                im2 = Image.fromarray(augmented["mask"])
                im1= im1.save(r"D:\Minor\leaf\new_data\train\image/"+ f'{str(name)}_{index}' + '.png')
                im2 = im2.save(r"D:\Minor\leaf\new_data\train\mask/"+ f'{str(name)}_{index}'+'.png')

                index += 1

        else:
            X = [x]
            Y = [y]

        
        


if __name__ == "__main__":
    """ Seeding """
# /    np.random.seed(42)

# #     """ Load the dataset """
#     data_path = r"D:\Minor\leaf\data"
#     (train_x, train_y), (test_x, test_y) = load_data(data_path)

#     print(f"Train:\t {len(train_x)} - {len(train_y)}")
#     print(f"Test:\t {len(test_x)} - {len(test_y)}")

    """ Data augmentation """
#     augment_data(train_x, train_y, "D:\Minor\leaf\new_data\train", augment=True)
#     augment_data(test_x, test_y, "D:\Minor\leaf\new_data\test", augment=False)
    
    

# Model

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, Reshape, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import tensorflow as tf

def SqueezeAndExcite(inputs, ratio=8):
    init = inputs
    filters = init.shape[-1]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    x = init * se
    return x

def ASPP(inputs):
    """ Image Pooling """
    shape = inputs.shape
    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(inputs)
    y1 = Conv2D(256, 1, padding="same", use_bias=False)(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)

    """ 1x1 conv """
    y2 = Conv2D(256, 1, padding="same", use_bias=False)(inputs)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    """ 3x3 conv rate=6 """
    y3 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=6)(inputs)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    """ 3x3 conv rate=12 """
    y4 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=12)(inputs)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    """ 3x3 conv rate=18 """
    y5 = Conv2D(256, 3, padding="same", use_bias=False, dilation_rate=18)(inputs)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])
    y = Conv2D(256, 1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def deeplabv3_plus(shape):
    """ Input """
    inputs = Input(shape)

    """ Encoder """
    encoder = ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)

    image_features = encoder.get_layer("conv4_block6_out").output
    x_a = ASPP(image_features)
    x_a = UpSampling2D((4, 4), interpolation="bilinear")(x_a)
#     x_a = tf.keras.layers.Reshape((270, 180,3))
    print(x_a)                              
    
    
    x_b = encoder.get_layer("conv2_block2_out").output
    x_b = Conv2D(filters=256, kernel_size=1, padding='same', use_bias=False)(x_b)
    x_b = BatchNormalization()(x_b)
    x_b = Activation('relu')(x_b)
    print(x_b)
#     return 0

    x = Concatenate()([x_a, x_b])
    x = SqueezeAndExcite(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=256, kernel_size=3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SqueezeAndExcite(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    x = Conv2D(1, 1)(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs, x)
    return model
    

if __name__ == "__main__":
    model = deeplabv3_plus((720,480,3))

KerasTensor(type_spec=TensorSpec(shape=(None, 180, 120, 256), dtype=tf.float32, name=None), name='up_sampling2d_1/resize/ResizeBilinear:0', description="created by layer 'up_sampling2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 180, 120, 256), dtype=tf.float32, name=None), name='activation_6/Relu:0', description="created by layer 'activation_6'")


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

# Train

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


""" Global parameters """
W = 720
H = 480

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(path):
    x = sorted(glob(os.path.join(path, "images", "*jpg")))
    y = sorted(glob(os.path.join(path, "masks", "*png")))
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch=1):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 40
    lr = 1e-4
    num_epochs = 10
    model_path = os.path.join("/home/jai201/Jai_Minor/", "deeplab_model.h5")
    csv_path = os.path.join("/home/jai201/Jai_Minor/", "data.csv")

    """ Dataset """
    dataset_path = r"/home/jai201/Jai_Minor/semantic_resized/split2"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "test")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch=batch_size)

#     """ Model """
    model = deeplabv3_plus((H,W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False),
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Train: 320 - 320
Valid: 80 - 80
KerasTensor(type_spec=TensorSpec(shape=(None, 120, 180, 256), dtype=tf.float32, name=None), name='up_sampling2d_7/resize/ResizeBilinear:0', description="created by layer 'up_sampling2d_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 120, 180, 256), dtype=tf.float32, name=None), name='activation_26/Relu:0', description="created by layer 'activation_26'")
Epoch 1/10


# Evaluation

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
# from metrics import dice_loss, dice_coef, iou
# from train import load_data

""" Global parameters """
H = 512
W = 512

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def save_results(image, mask, y_pred, save_image_path):
    ## i - m - yp - yp*i
    line = np.ones((H, 10, 3)) * 128

    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    mask = mask * 255

    y_pred = np.expand_dims(y_pred, axis=-1)    ## (512, 512, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)  ## (512, 512, 3)

    masked_image = image * y_pred
    y_pred = y_pred * 255

    cat_images = np.concatenate([image, line, mask, line, y_pred, line, masked_image], axis=1)
    cv2.imwrite(save_image_path, cat_images)

# if __name__ == "__main__":
    """ Seeding """
#     np.random.seed(42)
#     tf.random.set_seed(42)

#     """ Directory for storing files """
#     create_dir("results")

#     """ Loading model """
#     with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
#         model = tf.keras.models.load_model("/home/jai201/Jai_Minor/model.h5")

#     """ Load the dataset """
#     dataset_path = "/home/jai201/Jai_Minor/new_data"
#     valid_path = os.path.join(dataset_path, "test")
#     test_x, test_y = load_data(valid_path)
#     print(f"Test: {len(test_x)} - {len(test_y)}")

#     """ Evaluation and Prediction """
#     SCORE = []
#     for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
#         """ Extract the name """
#         name = x.split("/")[-1].split(".")[0]

#         """ Reading the image """
#         image = cv2.imread(x, cv2.IMREAD_COLOR)
#         x = image/255.0
#         x = np.expand_dims(x, axis=0)

#         """ Reading the mask """
#         mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)

#         """ Prediction """
#         y_pred = model.predict(x)[0]
#         y_pred = np.squeeze(y_pred, axis=-1)
#         y_pred = y_pred > 0.5
#         y_pred = y_pred.astype(np.int32)

#         """ Saving the prediction """
#         save_image_path = f"/home/jai201/Jai_Minor/results/{name}.png"
#         save_results(image, mask, y_pred, save_image_path)

#         """ Flatten the array """
#         mask = mask.flatten()
#         y_pred = y_pred.flatten()

#         """ Calculating the metrics values """
#         acc_value = accuracy_score(mask, y_pred)
#         f1_value = f1_score(mask, y_pred, pos_label='positive',average='micro')
#         jac_value = jaccard_score(mask, y_pred, pos_label='positive',average='micro')
#         recall_value = recall_score(mask, y_pred, pos_label='positive',average='micro')
#         precision_value = precision_score(mask, y_pred, pos_label='positive',average='micro')
#         SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

#     """ Metrics values """
#     score = [s[1:]for s in SCORE]
#     score = np.mean(score, axis=0)
#     print(f"Accuracy: {score[0]:0.5f}")
#     print(f"F1: {score[1]:0.5f}")
#     print(f"Jaccard: {score[2]:0.5f}")
#     print(f"Recall: {score[3]:0.5f}")
#     print(f"Precision: {score[4]:0.5f}")

#     df = pd.DataFrame(SCORE, columns=["Image", "Accuracy", "F1", "Jaccard", "Recall", "Precision"])
#     df.to_csv("files/score.csv")

# Prediction


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

""" Global parameters """
H = 512
W = 512

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """


    """ Loading model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("/home/jai201/Jai_Minor/model.h5")

    """ Load the dataset """
    data_x = glob("/home/jai201/Downloads/Leaf_Spot_Horizontal_15.jpg")

    for path in tqdm(data_x, total=len(data_x)):
        """ Extracting name """
        name = path.split("/")[-1].split(".")[0]

        """ Reading the image """
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        h, w, _ = image.shape
        x = cv2.resize(image, (W, H))
        x = x/255.0
        x = x.astype(np.float32)
        x = np.expand_dims(x, axis=0)

        """ Prediction """
        y = model.predict(x)[0]
        y = cv2.resize(y, (w, h))
        y = np.expand_dims(y, axis=-1)

        """ Save the image """
        masked_image = image * y
        line = np.ones((h, 10, 3)) * 128
        cat_images = np.concatenate([image, line, masked_image], axis=1)
        cv2.imwrite(f"/home/jai201/Jai_Minor/{name}.png", cat_images)